In [25]:
from functions.functions import *

from config import model

search_info_to_docs(model=model, question="Кто такой Дональд Трумп?", lng='ru')

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Donald_Trump', 'title': 'Donald Trump - Wikipedia', 'language': 'en'}, page_content='Donald Trump - WikipediaJump to contentMain menuMain menumove to sidebarhideNavigationMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usContributeHelpLearn to editCommunity portalRecent changesUpload fileSearchSearchAppearanceDonateCreate accountLog inPersonal toolsDonateCreate accountLog inPages for logged out editorslearn moreContributionsTalkContentsmove to sidebarhide(Top)1Early life and education2Personal lifeToggle Personal life subsection2.1Family2.2Health2.3Views3Business careerToggle Business career subsection3.1Real estate3.1.1Manhattan and Chicago developments3.1.2Atlantic City casinos3.1.3Clubs3.2Side ventures3.2.1Trump University3.3Foundation3.4Legal affairs and bankruptcies3.5Wealth4Media career5Early political aspirations (1987â€“2014)62016 presidential electionToggle 2016 presidential election subsection6.1

In [2]:
import re

text = """```["Donald Trump biography", "Donald Trump political career", "Donald Trump business career", "Donald Trump presidency", "Donald Trump family", "Donald Trump net worth", "Donald Trump controversies", "Donald Trump public image"]```"""

# Use regex to extract all strings inside the quotes
pattern = r'"(.*?)"'
result = re.findall(pattern, text)

print(type(result))

<class 'list'>


In [28]:
import shutil

shutil.rmtree("./db")

FileNotFoundError: [Errno 2] No such file or directory: './db'

In [1]:
import chromadb

chroma_client = chromadb.HttpClient(host='3.122.251.31', port=8000)

In [2]:
collection = chroma_client.get_or_create_collection("text")

In [ ]:
from chromadb.utils import embedding_functions

embedding_functions.SentenceTransformerEmbeddingFunction(model_name=)